In [137]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, monotonically_increasing_id
from pyspark.sql import SQLContext
#from pyspark.sql.types import 
from pyspark.sql.dataframe import *
from datetime import datetime
#from pyspark.sql.functions import timestamp
from pyspark.sql import types as T

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')


['dl.cfg']

In [3]:
config = configparser.ConfigParser()
config.read('dl.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS CREDS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS CREDS']['AWS_SECRET_ACCESS_KEY']

In [4]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [5]:
# def main():
#     spark = create_spark_session()
#     input_data = "s3a://udacity-dend/"
#     output_data = "s3a://data-lake-omkar/"
    
#     process_song_data(spark, input_data, output_data)    
#     process_log_data(spark, input_data, output_data)


In [8]:

spark = create_spark_session()

input_data = "s3a://udacity-dend/"

#song_data = os.path.join(input_data,"song_data/A/*/*/*.json")

df = spark.read.format("json").load("s3a://udacity-dend/song_data/A/A/A/TRAAAAK128F9318786.json")



In [10]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [22]:
df_songs=df.select("song_id","title","artist_id","year","duration")
df_songs.show()

+------------------+------+------------------+----+--------+
|           song_id| title|         artist_id|year|duration|
+------------------+------+------------------+----+--------+
|SOBLFFE12AF72AA5BA|Scream|ARJNIUY12298900C91|2009|213.9424|
+------------------+------+------------------+----+--------+



In [34]:
output="s3a://udacity-omkar"


parquetfile=df_songs.write.parquet("songs1.parquet")




In [46]:
parquetfilesongs=spark.read.parquet("songs1.parquet")

parquetfile.createOrReplaceTempView("parquetfile")

artist=spark.sql("select song_id,duration from parquetfile")

artist.show()


+------------------+--------+
|           song_id|duration|
+------------------+--------+
|SOBLFFE12AF72AA5BA|213.9424|
+------------------+--------+



In [53]:
parquetfilesongs=spark.read.parquet("songs1.parquet")
parquetfilesongs.write.partitionBy("year","artist_id").parquet("s3a://udacity-omkar1/parquet","overwrite")
                 


In [55]:
df_artist=df.select("artist_id","artist_name","artist_location","artist_latitude","artist_longitude")

df_artist.write.parquet("s3a://udacity-omkar1/artists")





In [64]:
log_data="s3a://udacity-dend/log_data/2018/11/*"

dflog=spark.read.format("json").load(log_data)

In [72]:
dflog=dflog.filter("Page=='NextSong'")





#dflog.select("Page").show()

In [73]:
dflog.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [74]:
users=dflog.select("userId","firstName","lastName","gender")

users.write.parquet("s3a://udacity-omkar1/users")



In [98]:
#dflog.select("ts").show()
dftime=dflog.select("ts")

timefunc=udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
datefunc=udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d'))

dfts=dftime.withColumn("ts",timefunc("ts"))
dftime.withColumn("ts",datefunc("ts")).show()

In [107]:
timedf=dfts.select(
            date_format('ts','h:m:s a ').alias("start_time"),
            hour('ts').alias('hour'),
            dayofmonth('ts').alias('day'),
            weekofyear('ts').alias('week'),
            month('ts').alias('month'),
            year('ts').alias('year'),
            date_format('ts','E').alias('weekday'))

timedf.write.partitionBy("year","month").parquet("s3a://udacity-omkar1/time")

In [113]:



input_data = "s3a://udacity-dend/song_data/A/A/A/*.json"
df=spark.read.format("json").load(input_data)



In [117]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [144]:
timefunc1=udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%H:%M:%S'))

songplays=df.join(dflog,(df.title==dflog.song) & (dflog.artist==df.artist_name) ).select( timefunc1('ts').alias('start_time'),"userId","level","song_id","artist_id","sessionId","location","userAgent")

In [148]:
#songplays=df.join(dflog,(df.title==dflog.song) ).select('ts',"userId","level","song_id","artist_id","sessionId","location","userAgent").show()

songplays=songplays.withColumn("songplay_id",monotonically_increasing_id())





In [149]:
songplays

root
 |-- start_time: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- location: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- songplay_id: long (nullable = false)

